In [41]:
import os 
import numpy as np
import pandas as pd 
from tqdm import tqdm
import pandas as pd
tqdm.pandas(desc='pandas bar')
# from pandarallel import pandarallel
import random
from tqdm.notebook import tqdm
import copy
import logging
# pandarallel.initialize(progress_bar=True)

import argparse
from types import SimpleNamespace
from FlagEmbedding import FlagModel

In [ ]:
### Here we transfer the data for Baseline in MTL solution

In [11]:
### Merge and Shuffle
amazon_google = pd.read_json('data/MoE/ER/amazon-google-train.json')
ant_buy = pd.read_json('data/MoE/ER/ant_buy_train_output.json')
semi_text_c = pd.read_json('data/MoE/ER/semi-text-c-train-MoE.json')
semi_text_w = pd.read_json('data/MoE/ER/semi-text-w-train-MoE.json')
walmart_amazon = pd.read_json('data/MoE/ER/walmart_amazon_train_output.json')
wdc_all = pd.read_json('data/MoE/ER/wdc_all_train_output.json')
hospital_train = pd.read_json('data/MoE/hospital-train-MoE.json')
beer_train = pd.read_json('data/MoE/beer-train-MoE.json')
rayyan_train = pd.read_json('data/MoE/rayyan-train-MoE.json')
RE = pd.read_json('data/RE/RE-train_t=4.json')
CMS_train = pd.read_json('data/MoE/SM/CMS_train_output.json')
mimic_train = pd.read_json('data/MoE/SM/mimic_train_output.json')
synthea_train = pd.read_json('data/MoE/SM/synthea_train_output.json')
walmart_train = pd.read_json('data/MoE/DI/walmart_train_output_wide.json')
amazon_train = pd.read_json('data/MoE/DI/amazon_train_output_wide.json')
restaurant_train = pd.read_json('data/MoE/DI/restaurant_train_output_wide.json')

In [349]:
amazon_google_train = pd.read_csv('LLM_ER/process_data/amazon-google/amazon_google_train_filter.csv',index_col=0)

In [53]:
def sample_dataframe_with_conditions(df,pos_num=100,neg_num=100):
    """
    Sample 200 rows from a DataFrame with specific conditions:
    - 100 rows with label=1, where (ltable, rtable) are unique.
    - 100 rows with label=0, where (ltable, rtable) are unique.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame with 'ltable', 'rtable', and 'label' columns.
    
    Returns:
    pd.DataFrame: A sampled DataFrame satisfying the conditions with the same structure as the input DataFrame.
    """
    # 分组数据框
    grouped_df = df.groupby('label')
    
    # 从label=1的组中选择100行，确保(ltable, rtable)互不相同
    sample_label_1 = grouped_df.get_group(1).groupby(['ltable_id', 'rtable_id']).apply(lambda x: x.sample(1))
    sample_label_1 = sample_label_1.head(pos_num)
    
    # 从label=0的组中选择100行，确保(ltable, rtable)互不相同
    sample_label_0 = grouped_df.get_group(0).groupby(['ltable_id', 'rtable_id']).apply(lambda x: x.sample(1))
    sample_label_0 = sample_label_0.head(neg_num)
    
    # 合并两组样本
    sampled_df = pd.concat([sample_label_1, sample_label_0])
    
    # 重置索引，确保索引是唯一的
    sampled_df.reset_index(drop=True, inplace=True)
    
    return sampled_df

In [19]:
amazon_google_fewshot = sample_dataframe_with_conditions(amazon_google_train)

In [350]:
tableA = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/tableA.csv').fillna('')
tableB = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/tableB.csv').fillna('')
tableA_dict = tableA.set_index('id').to_dict(orient='records')
tableB_dict = tableB.set_index('id').to_dict(orient='records')
train = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/train.csv')
valid = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/valid.csv')
test = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/test.csv')

In [25]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(tableA_dict[ltable_id]),json.dumps(tableB_dict[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
amazon_google_fewshot_output = amazon_google_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
amazon_google_fewshot_output_test = test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 2293/2293 [00:00<00:00, 20877.47it/s]


In [352]:
import json,ast
df = amazon_google_train
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']

    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(tableA_dict[ltable_id]),json.dumps(tableB_dict[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(tableA_dict[ltable_id])
        RAG_dict['Entity_2'] = str(tableB_dict[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
amazon_google_fewshot_output = amazon_google_train.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')
amazon_google_fewshot_output_test = test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 2293/2293 [00:01<00:00, 2045.09it/s]


In [92]:
amazon_google_fewshot.to_csv('LLM_ER/process_data/baseline/amazon_google_fewshot.csv')
semi_text_w_fewshot.to_csv('LLM_ER/process_data/baseline/semi_text_w_fewshot.csv')
semi_text_c_train_fewshot.to_csv('LLM_ER/process_data/baseline/semi_text_c_train_fewshot.csv')
walmart_amazon_train_fewshot.to_csv('LLM_ER/process_data/baseline/walmart_amazon_train_fewshot.csv')

In [354]:
amazon_google_fewshot_output.columns = ['instruction','input','output']
amazon_google_fewshot_output_test.columns = ['instruction','input','output']
json.dump(amazon_google_fewshot_output.to_dict(orient='records'), open('data/MoE_no_meta/ER/amazon_google_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(amazon_google_fewshot_output_test.to_dict(orient='records'), open('data/MoE_no_meta/ER/amazon_google_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Semi-Text-W-Baseline

In [355]:
semi_text_w_test = pd.read_csv('LLM_ER/process_data/semi-text-w/semi-text-w/test.csv',header=None)
semi_text_w_test.columns = ['ltable_id','rtable_id','label']

In [356]:
semi_text_w_train = pd.read_csv('LLM_ER/process_data/semi-text-w/train_add.csv',index_col=0)

In [357]:
dict_ltable = pd.read_json('LLM_ER/process_data/semi-text-w/semi-text-w/left.json').fillna('')
dict_ltable['index'] = dict_ltable.index
dict_ltable = dict_ltable.set_index('index').to_dict(orient='records')

In [358]:
# dict_rtable = pd.read_table('LLM_ER/process_data/semi-text-w/semi-text-w/right.txt',encoding='utf-8')
entities = []
file_path = "LLM_ER/process_data/semi-text-w/semi-text-w/right.txt"
with open(file_path, "r",encoding='utf-8') as rd:
    lines = rd.readlines()
    for line in tqdm(lines, desc="read text entity..."):
        text = line.strip()
        entities.append(text)
dict_rtable = entities

read text entity...:   0%|          | 0/9234 [00:00<?, ?it/s]

In [54]:
semi_text_w_fewshot = sample_dataframe_with_conditions(semi_text_w_train,pos_num=80,neg_num=160)

In [64]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
semi_text_w_fewshot_output = semi_text_w_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
semi_text_w_test_output = semi_text_w_test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar:   0%|          | 0/1846 [00:00<?, ?it/s]

pandas bar: 100%|██████████| 1846/1846 [00:00<00:00, 14811.19it/s]


In [360]:
import json,ast
df = semi_text_w_train
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
    # return text + RAG_text, '' , str(output_dict)
semi_text_w_fewshot_output = semi_text_w_train.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')
semi_text_w_test_output = semi_text_w_test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 1846/1846 [00:01<00:00, 1705.40it/s]


In [361]:
train_df = semi_text_w_fewshot_output
test_df = semi_text_w_test_output
train_df.columns = ['instruction','input','output']
test_df.columns = ['instruction','input','output']
json.dump(train_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/semi_text_w_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/semi_text_w_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [362]:
### Semi-Text-C
semi_text_c_train = pd.read_csv('LLM_ER/process_data/semi-text-c/train_add.csv',index_col=0)
semi_text_c_test = pd.read_csv('LLM_ER/process_data/semi-text-c/semi-text-c/test.csv',header=None)
dict_ltable = pd.read_json('LLM_ER/process_data/semi-text-c/semi-text-c/left.json').fillna('')
dict_ltable['index'] = dict_ltable.index
dict_ltable = dict_ltable.set_index('index').to_dict(orient='records')
entities = []
file_path = "LLM_ER/process_data/semi-text-c/semi-text-c/right.txt"
with open(file_path, "r",encoding='utf-8') as rd:
    lines = rd.readlines()
    for line in tqdm(lines, desc="read text entity..."):
        text = line.strip()
        entities.append(text)
dict_rtable = entities

read text entity...:   0%|          | 0/20897 [00:00<?, ?it/s]

In [363]:
semi_text_c_train.columns = ['ltable_id','rtable_id','label']
semi_text_c_test.columns = ['ltable_id','rtable_id','label']


In [78]:
semi_text_c_train_fewshot = sample_dataframe_with_conditions(semi_text_c_train,pos_num=80,neg_num=160)

In [81]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
semi_text_c_fewshot_output = semi_text_c_train_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
semi_text_c_test_output = semi_text_c_test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4179/4179 [00:00<00:00, 18401.90it/s]


In [364]:
import json,ast
df = semi_text_c_train
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
semi_text_c_fewshot_output = semi_text_c_train.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')
semi_text_c_test_output = semi_text_c_test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4179/4179 [00:02<00:00, 2031.87it/s]


In [365]:
train_df = semi_text_c_fewshot_output
test_df = semi_text_c_test_output
train_df.columns = ['instruction','input','output']
test_df.columns = ['instruction','input','output']
json.dump(train_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/semi_text_c_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/semi_text_c_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Walmart-Amazon-Baseline


In [366]:
walmart_amazon_train = pd.read_csv('LLM_ER/process_data/walmart-amazon/train.csv',index_col=0)
walmart_amazon_test = pd.read_csv('LLM_ER/process_data/walmart-amazon/test_align.csv',index_col=0)

In [375]:
tableA = pd.read_csv('LLM_ER/process_data/walmart-amazon/exp_data/tableA.csv').fillna('')
tableB = pd.read_csv('LLM_ER/process_data/walmart-amazon/exp_data/tableB.csv').fillna('')
dict_ltable = tableA.set_index('id').to_dict(orient='records')
dict_rtable = tableB.set_index('id').to_dict(orient='records')
# train = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/train.csv')
# valid = pd.read_csv('LLM_ER/process_data/amazon-google/amazon-google/valid.csv')
test = pd.read_csv('LLM_ER/process_data/walmart-amazon/exp_data/test.csv')

In [91]:
walmart_amazon_train_fewshot = sample_dataframe_with_conditions(walmart_amazon_train,pos_num=100,neg_num=100)

In [95]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
walmart_amazon_train_fewshot_output = walmart_amazon_train_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
walmart_amazon_test = walmart_amazon_test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 2049/2049 [00:00<00:00, 15684.78it/s]


In [376]:
import json,ast
df = walmart_amazon_train
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
walmart_amazon_train_fewshot_output = walmart_amazon_train.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')

# walmart_amazon_test.columns = ['ltable_id','rtable_id','label']
walmart_amazon_test = test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 2049/2049 [00:01<00:00, 1925.55it/s]


In [378]:
train_df = walmart_amazon_train_fewshot_output
test_df = walmart_amazon_test
train_df.columns = ['instruction','input','output']
test_df.columns = ['instruction','input','output']
json.dump(train_df.to_dict(orient='records'), open('data//MoE_no_meta/ER/walmart_amazon_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test_df.to_dict(orient='records'), open('data//MoE_no_meta/ER/walmart_amazon_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### wdc-all baseline

In [379]:
wdc_all_train_sample = pd.read_csv('LLM_ER/process_data/wdc-all/train_unique.csv',index_col=0)
wdc_all_test = pd.read_csv('LLM_ER/process_data/wdc-all/wdc_all/dpo_test.csv',index_col=0)

In [380]:
wdc_train = pd.read_csv('LLM_ER/process_data/wdc-all/all/train.txt.small',sep='\t',header=None)
wdc_valid = pd.read_csv('LLM_ER/process_data/wdc-all/all/valid.txt.small',sep='\t',header=None)
wdc_test = pd.read_csv('LLM_ER/process_data/wdc-all/all/test.txt',sep='\t',header=None)
wdc_train.columns = ['l_entity','r_entity','label']
wdc_valid.columns = ['l_entity','r_entity','label']
wdc_test.columns = ['l_entity','r_entity','label']
def ExtractEntity(row):
    l_entity = row[0].replace('COL title VAL ','').strip()
    r_entity = row[1].replace('COL title VAL ','').strip()
    return l_entity,r_entity,row[2]
wdc_train_out = wdc_train.apply(ExtractEntity,axis=1,result_type='expand')
wdc_valid_out = wdc_valid.apply(ExtractEntity,axis=1,result_type='expand')
wdc_test_out = wdc_test.apply(ExtractEntity,axis=1,result_type='expand')
wdc_all_origin = pd.concat([wdc_train_out,wdc_valid_out,wdc_test_out]).reset_index(drop=True)

In [381]:
wdc_all_origin['index'] = wdc_all_origin.index

In [382]:
dict_ltable = dict(zip(wdc_all_origin.index,wdc_all_origin[0]))
dict_rtable = dict(zip(wdc_all_origin.index,wdc_all_origin[1]))

In [112]:
wdc_all_train_fewshot = sample_dataframe_with_conditions(wdc_all_train_sample,pos_num=100,neg_num=100)

In [114]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    return text + RAG_text, '' , str(output_dict)
wdc_all_train_fewshot_output = wdc_all_train_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
wdc_all_test_output = wdc_all_test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4398/4398 [00:00<00:00, 20851.47it/s]


In [385]:
import json,ast
df = wdc_all_train_sample
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
wdc_all_train_fewshot_output = wdc_all_train_sample.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')
wdc_all_test_output = wdc_all_test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 4398/4398 [00:02<00:00, 1867.13it/s]


In [387]:
train_df = wdc_all_train_fewshot_output
test_df = wdc_all_test_output
train_df.columns = ['instruction','input','output']
test_df.columns = ['instruction','input','output']
json.dump(train_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/wdc_all_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/wdc_all_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Ant-Buy Baseline

In [388]:
ant_buy_train = pd.read_csv('LLM_ER/process_data/ant-buy/train.csv',index_col=0)
ant_buy_test = pd.read_csv('LLM_ER/process_data/ant-buy/exp_data/dpo_test.csv',index_col=0)

In [389]:
tableA = pd.read_csv('LLM_ER/process_data/ant-buy/exp_data/tableA.csv').fillna('')
tableB = pd.read_csv('LLM_ER/process_data/ant-buy/exp_data/tableB.csv').fillna('')
dict_ltable = tableA.set_index('id').to_dict(orient='records')
dict_rtable = tableB.set_index('id').to_dict(orient='records')

In [118]:
ant_buy_train_fewshot = sample_dataframe_with_conditions(ant_buy_train,pos_num=100,neg_num=100)

In [ ]:
import json,ast
def Prompt_Generalize_Baseline(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
ant_buy_train_fewshot_output = ant_buy_train_fewshot.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')
ant_buy_test_output = ant_buy_test.progress_apply(Prompt_Generalize_Baseline,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

In [391]:
import json,ast
df = ant_buy_train
def Prompt_Generalize_Baseline_NoMeta(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    temp_dict = {}
    temp_dict['Output'] = ''
    RAG_text = ''
    ltable_id = row['ltable_id']
    rtable_id = row['rtable_id']
    text = 'You are an expert in detecting if two text descriptions of entities refer to the same or different entities. You are well-known to examine the fine details and can find subtle matches or mismatches to help you arrive at the final judgement. \n\nJudge whether Entity 1 and Entity 2 are match or dismatch, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [match,dismatch]\n\nOutput format example:%s\n\nEntity 1:%s\n\nEntity 2:%s' % (json.dumps(temp_dict),json.dumps(dict_ltable[ltable_id]),json.dumps(dict_rtable[rtable_id]))
    output_dict = {}
    output_dict['Output'] = 'dismatch' if row['label']==0 else 'match'
    sample_df = df.sample(n=5)
    RAG_text += '\n\nTake these examples as reference:\n\n'
    for index,row_ in sample_df.iterrows():
        ltable_id = row_['ltable_id']
        rtable_id = row_['rtable_id']
        RAG_dict = {}
        RAG_dict['Entity_1'] = str(dict_ltable[ltable_id])
        RAG_dict['Entity_2'] = str(dict_rtable[rtable_id])
        RAG_dict['Output'] = 'dismatch' if row_['label']==0 else 'match'
        RAG_text += '%s\n\n' % str(RAG_dict)
    return text + RAG_text, '' , str(output_dict)
ant_buy_train_fewshot_output = ant_buy_train.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')
ant_buy_test_output = ant_buy_test.progress_apply(Prompt_Generalize_Baseline_NoMeta,axis=1,result_type='expand')           
# amazon_google_output = amazon_google_test.progress_apply(Prompt_Generalize,axis=1,result_type='expand')           

pandas bar: 100%|██████████| 1916/1916 [00:01<00:00, 1890.88it/s]


In [392]:
train_df = ant_buy_train_fewshot_output
test_df = ant_buy_test_output
train_df.columns = ['instruction','input','output']
test_df.columns = ['instruction','input','output']
json.dump(train_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/ant_buy_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/ant_buy_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [400]:
import os

def list_csv_files_and_parents(folder_path):
    """
    List all CSV files in the specified folder and their parent folder names.

    Parameters:
    folder_path (str): The path to the directory.

    Returns:
    list: A list where each element is a list containing the parent folder name and CSV file path.
    """
    csv_files_info = []

    # 遍历指定路径内的所有文件和文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".json") and file.__contains__('train'):
                # 获取CSV文件的路径
                csv_file_path = os.path.join(root, file)
                # 获取CSV文件所在一级父文件夹名字
                parent_folder_name = os.path.basename(root)
                # 将父文件夹名字和CSV文件路径添加到列表中
                csv_files_info.append('/'.join(csv_file_path.split('/')[-3:]))

    return csv_files_info
MoE_noMeta = list_csv_files_and_parents('data/MoE_no_meta/ER/')

In [38]:
def SHA1_cal(path):
    hexdigest = encrypt(path, 'sha1')
    return hexdigest

In [ ]:
SHA

In [407]:
dataset_info = {}
def SHA1_cal(path):
    hexdigest = encrypt(path, 'sha1')
    return hexdigest
# SHA1_cal('data/MoE/ER/ant_buy_train_output.json')
for dataset in MoE_noMeta:
# for i in [6,7,8]:
    name = dataset.split('/')[-1].split('.')[0]
    print(name)
    # path_save = 'data/MoE/add/%s.json' % name
    name += '-MoE-noMeta'
    dataset_info[name] = {}
    dataset_info[name]["file_name"] = dataset
    dataset_info[name]["file_sha1"] = SHA1_cal('data/' + dataset)
    dataset_info[name]["columns"] = {
        "prompt": "instruction",
        "query": "input",
        "response": "output"
    }

amazon_google_train
semi_text_w_train
semi_text_c_train
walmart_amazon_train
wdc_all_train
ant_buy_train


In [408]:
str(dataset_info).replace("'",'"')[1:-1]

'"amazon_google_train-MoE-noMeta": {"file_name": "MoE_no_meta/ER/amazon_google_train.json", "file_sha1": "c94be84eaf41c96cffab1a1e399fc088b78d6c57", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "semi_text_w_train-MoE-noMeta": {"file_name": "MoE_no_meta/ER/semi_text_w_train.json", "file_sha1": "6a954eec583781894e80e37056388e395f3140ec", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "semi_text_c_train-MoE-noMeta": {"file_name": "MoE_no_meta/ER/semi_text_c_train.json", "file_sha1": "2d9641a34c8b02eab03a83568a5cc50224030a94", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "walmart_amazon_train-MoE-noMeta": {"file_name": "MoE_no_meta/ER/walmart_amazon_train.json", "file_sha1": "c1ee9a10cc9a3b53b879d5bab35220a6431674f8", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "wdc_all_train-MoE-noMeta": {"file_name": "MoE_no_meta/ER/wdc_all_train.json", "file_sha1": "ea65

In [ ]:

for i in range(len(merge_df_output)):
# for i in [6,7,8]:
    df = merge_df_output[i]
    name = find_var_name(merge_df_input[i])
    print(name)
    path_save = 'data/MoE/add/%s.json' % name
    json.dump(df.to_dict(orient='records'), open(path_save, 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    name += '-MoE-Add'
    dataset_info[name] = {}
    dataset_info[name]["file_name"] = path_save.replace('data/','')
    dataset_info[name]["file_sha1"] = SHA1_cal(path_save)
    dataset_info[name]["columns"] = {
        "prompt": "instruction",
        "query": "input",
        "response": "output"
    }

In [410]:
["WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset %s     --output_dir lora_weight/MoE-no-Meta/%s --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 5.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16     --template mistral     --lora_r 16 --logging_steps 5 --plot_loss  --lora_target all --save_steps 50 --use_unsloth --cutoff_len 2048 --quantization_bit 8" % (t,t) for t in dataset_info.keys()][1:]

['WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset semi_text_w_train-MoE-noMeta     --output_dir lora_weight/MoE-no-Meta/semi_text_w_train-MoE-noMeta --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 5.0     --gradient_accumulation_steps 8     --per_device_eval_batch_size 8     --fp16     --template mistral     --lora_r 16 --logging_steps 5 --plot_loss  --lora_target all --save_steps 50 --use_unsloth --cutoff_len 2048 --quantization_bit 8',
 'WANDB_MODE=disabled accelerate launch src/train_bash.py     --stage sft     --model_name_or_path Mistral-7B-Instruct-v0.2    --do_train     --finetuning_type lora     --dataset semi_text_c_train-MoE-noMeta     --output_dir lora_weight/MoE-no-Meta/semi_text_c_train-MoE-noMeta --overwrite_output_dir     --lr_scheduler_type cosine     --num_train_epochs 5.0     --gradient_accumulation_steps 8     -

## Data Cleaning Baseline Data


In [125]:
hospital_train_baseline = pd.read_json('MoE-Example/DC/hospital-train-ablation.json').iloc[:200]
beer_train_baseline = pd.read_json('MoE-Example/DC/beer-train-ablation.json').iloc[:200]
rayyan_baseline = pd.read_json('MoE-Example/DC/rayyan-train-ablation.json').iloc[:200]


In [128]:
json.dump(hospital_train_baseline.to_dict(orient='records'), open('data/MoE_Baseline/DC/hospital_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(beer_train_baseline.to_dict(orient='records'), open('data/MoE_Baseline/DC/beer_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(rayyan_baseline.to_dict(orient='records'), open('data/MoE_Baseline/DC/rayyan_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

## Schema Matching Baseline Data

In [253]:
CMS_test = pd.read_json('data/MoE/SM/CMS_test_output.json')
mimic_test = pd.read_json('data/MoE/SM/mimic_test_output.json')
synthea_test = pd.read_json('data/MoE/SM/synthea_test_output.json')

In [139]:

CMS_train_baseline = pd.concat([CMS_train[CMS_train['output']=="{'Output': 'match'}"],CMS_train[CMS_train['output']=="{'Output': 'dismatch'}"].sample(n=500)])
CMS_train_baseline['instruction'] = CMS_train_baseline.progress_apply(Remove_RAG,axis=1)

pandas bar: 100%|██████████| 660/660 [00:00<00:00, 33126.00it/s]


In [142]:
mimic_train_baseline = pd.concat([mimic_train[mimic_train['output']=="{'Output': 'match'}"],mimic_train[mimic_train['output']=="{'Output': 'dismatch'}"].sample(n=500)])
mimic_train_baseline['instruction'] = mimic_train_baseline.progress_apply(Remove_RAG,axis=1)

pandas bar: 100%|██████████| 599/599 [00:00<00:00, 23150.74it/s]


In [143]:
synthea_train_baseline = pd.concat([synthea_train[synthea_train['output']=="{'Output': 'match'}"],synthea_train[synthea_train['output']=="{'Output': 'dismatch'}"].sample(n=500)])
synthea_train_baseline['instruction'] = synthea_train_baseline.progress_apply(Remove_RAG,axis=1)

pandas bar: 100%|██████████| 584/584 [00:00<00:00, 26047.70it/s]


In [255]:
CMS_test['instruction'] = CMS_test.progress_apply(Remove_RAG,axis=1)
mimic_test['instruction'] = mimic_test.progress_apply(Remove_RAG,axis=1)
synthea_test['instruction'] = synthea_test.progress_apply(Remove_RAG,axis=1)

pandas bar: 100%|██████████| 5928/5928 [00:00<00:00, 117964.99it/s]


In [144]:
json.dump(CMS_train_baseline.to_dict(orient='records'), open('data/MoE_Baseline/SM/CMS_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(synthea_train_baseline.to_dict(orient='records'), open('data/MoE_Baseline/SM/synthea_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(mimic_train_baseline.to_dict(orient='records'), open('data/MoE_Baseline/SM/mimic_train_baseline.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [258]:
json.dump(CMS_test.to_dict(orient='records'), open('data/MoE_Baseline/SM/CMS_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(mimic_test.to_dict(orient='records'), open('data/MoE_Baseline/SM/mimic_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(synthea_test.to_dict(orient='records'), open('data/MoE_Baseline/SM/synthea_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

## DI

In [157]:
walmart_train_fewshot = walmart_train.iloc[:241]
amazon_train_fewshot = amazon_train.iloc[:2001]
restaurant_train_fewshot = restaurant_train.iloc[:86]

In [158]:
def Data_Imputation_Remove_RAG_Option(row):
    instruction = row['instruction']
    instruction_1st = instruction.split('\n\nOptions')[0]
    instruction_2nd = '\n\n'+instruction.split(']\n\n')[1].split('\n\nTake these examples as reference')[0]
    return instruction_1st + instruction_2nd
# print(Data_Imputation_Remove_RAG_Option(walmart_train.iloc[0]))
walmart_train_fewshot['instruction'] = walmart_train_fewshot.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)
amazon_train_fewshot['instruction'] = amazon_train_fewshot.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)
restaurant_train_fewshot['instruction'] = restaurant_train_fewshot.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)

pandas bar: 100%|██████████| 241/241 [00:00<00:00, 21396.34it/s]
/tmp/ipykernel_1942508/3893300183.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  walmart_train_fewshot['instruction'] = walmart_train_fewshot.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)
pandas bar: 100%|██████████| 2001/2001 [00:00<00:00, 31933.89it/s]
/tmp/ipykernel_1942508/3893300183.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_train_fewshot['instruction'] = amazon_train_fewshot.progress_apply(Data_Imputation_

In [159]:
walmart_test = pd.read_json('data/MoE/DI/walmart_test_output_wide.json')
amazon_test = pd.read_json('data/MoE/DI/amazon_test_output_wide.json')
restaurant_test = pd.read_json('data/MoE/DI/restaurant_test_output_wide.json')

In [160]:
walmart_test['instruction'] = walmart_test.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)
amazon_test['instruction'] = amazon_test.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)
restaurant_test['instruction'] = restaurant_test.progress_apply(Data_Imputation_Remove_RAG_Option,axis=1)

pandas bar: 100%|██████████| 29/29 [00:00<00:00, 11917.97it/s]


In [161]:
json.dump(walmart_train_fewshot.to_dict(orient='records'), open('data/MoE_Baseline/DI/walmart_train_fewshot.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(amazon_train_fewshot.to_dict(orient='records'), open('data/MoE_Baseline/DI/amazon_train_fewshot.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(restaurant_train_fewshot.to_dict(orient='records'), open('data/MoE_Baseline/DI/restaurant_train_fewshot.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [163]:
json.dump(walmart_test.to_dict(orient='records'), open('data/MoE_Baseline/DI/walmart_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(amazon_test.to_dict(orient='records'), open('data/MoE_Baseline/DI/amazon_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(restaurant_test.to_dict(orient='records'), open('data/MoE_Baseline/DI/restaurant_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Start AVE Example

In [188]:
oa_mine = pd.read_json('data/oa_mine/oa_mine_train_small.json')
oa_mine_test = pd.read_json('data/oa_mine/oa_mine_test_small.json')

In [182]:
oa_mine_fewshot = oa_mine

In [183]:
def AVE_Remove_RAG_Option(row):
    instruction = row['instruction']
    # instruction_1st = instruction.split('\n\nOptions')[0]
    instruction_2nd = instruction.split('\n\nTake these')[0]
    return instruction_2nd

In [184]:
oa_mine_fewshot['instruction'] = oa_mine_fewshot.progress_apply(AVE_Remove_RAG_Option,axis=1)

pandas bar: 100%|██████████| 1443/1443 [00:00<00:00, 35523.04it/s]


In [189]:
oa_mine_test['instruction'] = oa_mine_test.progress_apply(AVE_Remove_RAG_Option,axis=1)

pandas bar: 100%|██████████| 2451/2451 [00:00<00:00, 36020.59it/s]


In [190]:
# json.dump(oa_mine_fewshot.to_dict(orient='records'), open('data/MoE_Baseline/AVE/oa_mine_fewshot_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(oa_mine_test.to_dict(orient='records'), open('data/MoE_Baseline/AVE/oa_mine_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [ ]:
### Start CTA Here

In [195]:
SimTab_train = pd.read_json('data/CTA/SimTab_Train_few.json')
SimTab_relation = np.load('data/CTA/sim_all_relation.npy',allow_pickle=True)

In [199]:
SimTab_test = pd.read_json('data/CTA/SimTab_test_few.json')

In [196]:
def SimTab_remove_RAG(row):
    instruction = row['instruction']
    ins_1 = instruction.split('Options:')[0] + 'Options:%s' % str(SimTab_relation)
    ins_2 = '\n\n'+instruction.split(']\n\n')[1].split('\n\nReference tables:')[0] 
    return ins_1 + ins_2
SimTab_train['instruction'] = SimTab_train.progress_apply(SimTab_remove_RAG,axis=1)

pandas bar: 100%|██████████| 6369/6369 [00:02<00:00, 2427.08it/s]


In [200]:
SimTab_test['instruction'] = SimTab_test.progress_apply(SimTab_remove_RAG,axis=1)

pandas bar: 100%|██████████| 7610/7610 [00:03<00:00, 2416.73it/s]


In [260]:
webtable_train = pd.read_json('data/CTA/WebTable_Train_few.json')
webtable_relation = np.load('data/CTA/webtable_all_relation.npy',allow_pickle=True)
webtable_test = pd.read_json('data/CTA/WebTable_Test_few.json')

In [261]:
print(webtable_test.iloc[0,0])

You are an expert in relation extraction from wikipedia web table to knowledge graph. Given table title and column pair for Table 1, please choose the most proper type from the provided options. Return in json format.

Column: col_1

Options:["city", "birthplace", "location"]

Output Format Example:

{"type": ""}

Table 1:

{'col_1': 'Reno, NV'}
{'col_1': 'Logan, UT'}

Reference tables:

{'Table': '{\'description\': \'"Schools Out" Camp\', \'col_1\': \'Activity & Rec Ctr\', \'status\': \'Unavailable\', \'gender\': \'Co-ed\'}', 'Column': 'col_1', 'type': 'location'}

{'Table': "{'col_1': 'Philadelphia', 'state': 'PA'}\n{'col_1': 'Fayetteville', 'state': 'AR'}\n{'col_1': 'New York', 'state': 'NY'}\n{'col_1': 'Rochester', 'state': 'NY'}", 'Column': 'col_1', 'type': 'city'}

{'Table': "{'name': 'Jacob Markstrom', 'weight': '196', 'age': '25', 'col_1': 'Gävle, SWE'}\n{'name': 'Ryan Miller', 'weight': '168', 'age': '35', 'col_1': 'East Lansing, MI, USA'}", 'Column': 'col_1', 'type': 'birthpl

In [262]:
def webtable_remove_RAG(row):
    instruction = row['instruction']
    ins_1 = instruction.split('Options:')[0] + 'Options:%s' % str(webtable_relation)
    ins_2 = '\n\n'+instruction.split(']\n\n')[1].split('\n\nReference tables:')[0] 
    return ins_1 + ins_2
webtable_test['instruction'] = webtable_test.progress_apply(webtable_remove_RAG,axis=1)

pandas bar: 100%|██████████| 17709/17709 [00:02<00:00, 7548.56it/s]


In [214]:
webtable_train['instruction'] = webtable_train.progress_apply(webtable_remove_RAG,axis=1)
webtable_test['instruction'] = webtable_test.progress_apply(webtable_remove_RAG,axis=1)

pandas bar: 100%|██████████| 17709/17709 [00:02<00:00, 7859.19it/s]


In [266]:
webtable_train['instruction'] = webtable_train.progress_apply(webtable_remove_RAG,axis=1)

pandas bar: 100%|██████████| 17772/17772 [00:02<00:00, 7550.10it/s]


In [201]:
json.dump(SimTab_train.to_dict(orient='records'), open('data/MoE_Baseline/CTA/SimTab_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(SimTab_test.to_dict(orient='records'), open('data/MoE_Baseline/CTA/SimTab_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [268]:
json.dump(webtable_train.to_dict(orient='records'), open('data/MoE_Baseline/CTA/webtable_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
# json.dump(webtable_test.to_dict(orient='records'), open('data/MoE_Baseline/CTA/webtable_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

## Relation Extraction

In [218]:
re_train = pd.read_json('data/RE/RE-train-no-ret.json')
re_test = pd.read_json('data/RE/RE-test-no-ret.json')

In [40]:
import hashlib


def encrypt(fpath: str, algorithm: str) -> str:
    with open(fpath, 'rb') as f:
        return hashlib.new(algorithm, f.read()).hexdigest()


if __name__ == '__main__':
    for algorithm in ('md5', 'sha1', 'sha256'):
        hexdigest = encrypt('data/MoE_Baseline/ED/error_detection_train.json', algorithm)
        print(f'{algorithm}: {hexdigest}')

md5: df42054726d765b7dbb449a566f74774
sha1: 471684456fc9e02903d71151ace37d45dc35be93
sha256: 4dc3daa77ff784e29c963c33a18a2d49e3d49584042eb255cc075a02af873b31


In [341]:
from transformers import RobertaTokenizer
import pandas as pd
from tqdm.notebook import tqdm
prompt_opt = pd.read_json('data/hospital/hospital-test.json') ## replace with your training file
# prompt_opt = SimTab_train
tokenizer = RobertaTokenizer.from_pretrained('roberta-base') ## replace with any LM


token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in tqdm(prompt_opt['instruction'].to_list())]

print("Token Lengths:", max(token_lengths))

  0%|          | 0/508 [00:00<?, ?it/s]

Token Lengths: 714


In [342]:
sum(token_lengths)

325112

In [221]:
import os
import pandas as pd

def concatenate_json_files_recursive(folder_path):
    """
    Concatenate all JSON files in the given folder and its subfolders that have 'train' in their filenames into a single DataFrame.

    Parameters:
    folder_path (str): The path to the root folder.

    Returns:
    pd.DataFrame: A concatenated DataFrame.
    """
    # 初始化空DataFrame，用于存储拼接的数据
    concatenated_df = pd.DataFrame()

    # 使用os.walk遍历所有子文件夹
    for root, dirs, files in os.walk(folder_path):
        # 获取当前文件夹下所有文件名含有'train'的json文件
        json_files = [file for file in files if 'train' in file and file.endswith('.json')]
        
        # 遍历当前文件夹下的json文件
        for json_file in json_files:
            file_path = os.path.join(root, json_file)
            with open(file_path, 'r', encoding='utf-8') as file:
                # 读取json文件并转换为DataFrame
                json_data = pd.read_json(file)
                # 拼接到总的DataFrame
                concatenated_df = pd.concat([concatenated_df, json_data])

    # 重置索引以确保索引是唯一的
    concatenated_df.reset_index(drop=True, inplace=True)

    return concatenated_df
concate_train = concatenate_json_files_recursive('data/MoE_Baseline')

In [223]:
json.dump(concate_train.to_dict(orient='records'), open('data/MoE_Baseline/concate_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [225]:
df_1 = pd.read_json('data/MoE_Baseline/CTA/SimTab_train.json')
df_2 = pd.read_json('data/MoE_Baseline/CTA/webtable_train.json')
df_3 = pd.read_json('data/MoE_Baseline/RE/RE-train.json')
Table_Baseline = pd.concat([df_1,df_2,df_3])

In [227]:
json.dump(Table_Baseline.to_dict(orient='records'), open('data/MoE_Baseline/concate_train_table.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [229]:
len(Table_Baseline)

26281

In [ ]:
### Merging Data

In [ ]:
### Merging for CTA-SimTab

In [230]:
SimTab_train_init = pd.read_json('data/CTA/SimTab_Train_few.json')
WebTable_train_init = pd.read_json('data/CTA/WebTable_Train_few.json')
Amazon_google_init = pd.read_json('data/MoE/ER/amazon-google-train.json')
Semi_text_w_init = pd.read_json('data/MoE/ER/semi-text-w-train-MoE.json')
Walmart_init = pd.read_json('data/MoE/DI/walmart_train_output_wide.json')
restaurant_init = pd.read_json('data/MoE/DI/restaurant_train_output_wide.json')

In [243]:
import pandas as pd


def merge_dataframes(dataframes):
    merged_dataframes = []

    for i, df in enumerate(dataframes):
        # 创建一个空的DataFrame用于累积合并的结果
        cumulative_merge = pd.DataFrame()

        for j, other_df in enumerate(dataframes):
            if i != j:
                # 根据output列中的元素种类执行不同的合并策略
                if other_df['output'].nunique() == 2:
                    # 执行原始策略
                    dismatch_rows = other_df[other_df['output'].str.contains('dismatch', na=False)]
                    non_dismatch_rows = other_df[~other_df['output'].str.contains('dismatch', na=False)]

                    selected_dismatch = dismatch_rows.sample(n=min(100, len(dismatch_rows)), replace=True)
                    selected_non_dismatch = non_dismatch_rows.sample(n=min(300, len(non_dismatch_rows)), replace=True)

                    merge_result = pd.concat([selected_dismatch, selected_non_dismatch])
                else:
                    # 随机选择150行数据
                    selected_rows = other_df.sample(n=min(300, len(other_df)), replace=True)
                    merge_result = selected_rows

                # 累积合并结果
                cumulative_merge = pd.concat([cumulative_merge, merge_result])

        # 将累积的合并结果与原始DataFrame合并
        # final_merged_df = pd.concat([df, cumulative_merge])
        # 不将累积的合并结果与原始DataFrame合并
        final_merged_df = pd.concat([cumulative_merge])
        # 将最终合并的DataFrame添加到结果列表中
        merged_dataframes.append(final_merged_df)

    return merged_dataframes

merge_df_output = merge_dataframes([SimTab_train_init.iloc[:,:3],WebTable_train_init.iloc[:,:3],Amazon_google_init,Semi_text_w_init,Walmart_init,restaurant_init])

In [247]:
def SHA1_cal(path):
    hexdigest = encrypt(path, 'sha1')
    return hexdigest
def find_var_name(obj):
    for var_name in globals():
        if globals()[var_name] is obj:
            return var_name
    return None
# SHA1_cal('data/MoE/ER/ant_buy_train_output.json')

In [ ]:
SimTab_train_init

In [249]:
merge_df_input = [SimTab_train_init,WebTable_train_init,Amazon_google_init,Semi_text_w_init,Walmart_init,restaurant_init]

In [250]:
import json
dataset_info = {}
for i in range(len(merge_df_output)):
# for i in [6,7,8]:
    df = merge_df_output[i]
    name = find_var_name(merge_df_input[i])
    print(name)
    path_save = 'data/MoE/add_CT/%s.json' % name
    json.dump(df.to_dict(orient='records'), open(path_save, 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    name += '-MoE-Add'
    dataset_info[name] = {}
    dataset_info[name]["file_name"] = path_save.replace('data/','')
    dataset_info[name]["file_sha1"] = SHA1_cal(path_save)
    dataset_info[name]["columns"] = {
        "prompt": "instruction",
        "query": "input",
        "response": "output"
    }

SimTab_train_init
WebTable_train_init
Amazon_google_init
Semi_text_w_init
Walmart_init
restaurant_init


In [251]:
str(dataset_info).replace("'",'"')[1:-1]

'"SimTab_train_init-MoE-Add": {"file_name": "MoE/add_CT/SimTab_train_init.json", "file_sha1": "5a2de2592a9a7eb4fe0c7cdf7dfb62b061a519f3", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "WebTable_train_init-MoE-Add": {"file_name": "MoE/add_CT/WebTable_train_init.json", "file_sha1": "47c71d2c00c88692661403a97aa6f7d99d5cab91", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "Amazon_google_init-MoE-Add": {"file_name": "MoE/add_CT/Amazon_google_init.json", "file_sha1": "2f6cb27493379fffb05c533af54d6f0faa24129e", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "Semi_text_w_init-MoE-Add": {"file_name": "MoE/add_CT/Semi_text_w_init.json", "file_sha1": "d1e4cfcc4a3603a4968eb5ac22ae7cd77f327131", "columns": {"prompt": "instruction", "query": "input", "response": "output"}}, "Walmart_init-MoE-Add": {"file_name": "MoE/add_CT/Walmart_init.json", "file_sha1": "47ea93bf5d4f04f9d0d6c18290225991913e3577", "col

In [252]:
task_list = list(dataset_info.keys())
task_list

['SimTab_train_init-MoE-Add',
 'WebTable_train_init-MoE-Add',
 'Amazon_google_init-MoE-Add',
 'Semi_text_w_init-MoE-Add',
 'Walmart_init-MoE-Add',
 'restaurant_init-MoE-Add']

In [ ]:
## Data Cleaning Evaluation

In [276]:
hospital_index = np.load('MoE-Example/DC/GEIL_Data/hospital/detector/detector.npy')

In [328]:
webtable_train_all = pd.read_json('data/MoE_Baseline/CTA/SimTab_train.json')

In [330]:
# def filter_df_by_occurrence(df, column_name, m):
#     """
#     Filter a DataFrame based on the occurrence of elements in a specific column.

#     Parameters:
#     df (pd.DataFrame): The input DataFrame.
#     column_name (str): The column name to consider for filtering.
#     m (int): The threshold for occurrence.

#     Returns:
#     pd.DataFrame: A filtered DataFrame with the same structure.
#     """
#     # 计算元素出现的次数
#     occurrence_counts = df[column_name].value_counts()

#     # 筛选出现次数小于等于m的元素
#     elements_to_keep = occurrence_counts[occurrence_counts <= m].index.tolist()

#     # 对出现次数大于m的元素，随机选择m个元素保留
#     elements_to_sample = occurrence_counts[occurrence_counts > m].index.tolist()
#     sampled_elements = np.random.choice(elements_to_sample, size=min(m, len(elements_to_sample)), replace=False)

#     # Convert sampled elements to a list of strings
#     sampled_elements = [str(element) for element in sampled_elements]

#     # 合并要保留的元素
#     elements_to_keep += sampled_elements

#     # 根据筛选条件过滤DataFrame
#     filtered_df = df[df[column_name].astype(str).isin(elements_to_keep)].copy()

#     return filtered_df
def shrink_df(df, m):
    """
    Shrink a DataFrame by randomly selecting m rows with unique values in the 'index' column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    m (int): The number of rows to select.

    Returns:
    pd.DataFrame: A shrunken DataFrame with the same structure.
    """
    # 获取'index'列的元素
    index_values = df['index'].values

    # 如果m大于当前DataFrame的行数，则返回整个DataFrame
    if m >= len(df):
        return df

    # 随机选择m个不重复的'index'值
    selected_indices = np.random.choice(index_values, size=m, replace=False)

    # 根据选取的'index'值筛选行
    shrunken_df = df[df['index'].isin(selected_indices)].copy()

    return shrunken_df
m = 20
df_sample = pd.DataFrame()
unique_value = list(webtable_train_all['output'].unique())
for u in unique_value:
    select = webtable_train_all[webtable_train_all['output']==u]
    if(len(select)<m):
        df_sample = pd.concat([df_sample,select])
    else:
        df_sample = pd.concat([df_sample,shrink_df(select,m)])
len(df_sample)

4093

In [331]:
json.dump(df_sample.to_dict(orient='records'), open('data/MoE_Baseline/CTA/SimTab_train_few.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### From here we combine the multiple MoE_no_Meta_Path test file for the convenience of batch inference with vLLM

In [2]:
import os
import pandas as pd
import json
from glob import glob

def concatenate_test_json_files(folder_path):
    # 构建一个路径模式来匹配所有的名字中包含'test'的json文件
    pattern = os.path.join(folder_path, '*test*.json')
    # 使用glob模块找到所有匹配的文件
    files = glob(pattern)
    
    # 初始化一个空的DataFrame用于之后的数据拼接
    concatenated_df = pd.DataFrame()
    
    # 遍历所有匹配的文件
    for file_path in files:
        # 读取json文件
        with open(file_path, 'r') as file:
            data = json.load(file)
            df = pd.DataFrame(data)
        
        # 获取文件名作为domain值，去掉路径和.json后缀
        domain = os.path.basename(file_path).replace('.json', '')
        # 添加domain列
        df['domain'] = domain
        
        # 将当前文件的DataFrame拼接到总的DataFrame中
        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
        
    return concatenated_df

# 使用示例
folder_path = 'data/MoE_no_meta/ER'
result_df = concatenate_test_json_files(folder_path)

In [4]:
json.dump(result_df.to_dict(orient='records'), open('data/MoE_no_meta/ER/all_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [8]:
hospital_detection = pd.read_csv('MoE-Example/GEIL_Data/hospital/detector/train.csv',index_col=0)
beer_detection = pd.read_csv('MoE-Example/GEIL_Data/beers/detector/train.csv',index_col=0)

rayyan_detection = pd.read_csv('MoE-Example/GEIL_Data/rayyan/detector/train.csv',index_col=0)


In [26]:
hospital_detection_test = pd.read_csv('MoE-Example/GEIL_Data/hospital/detector/test.csv',index_col=0)
beer_detection_test = pd.read_csv('MoE-Example/GEIL_Data/beers/detector/test.csv',index_col=0)

rayyan_detection_test = pd.read_csv('MoE-Example/GEIL_Data/rayyan/detector/test.csv',index_col=0)


In [ ]:
def parse_text_to_dict(text):
    # 分割字符串以"COL"为标志，忽略第一个空分割结果
    parts = text.split("COL")[1:]
    result_dict = {}
    
    for part in parts:
        # 每个部分以"VAL"分割为键和值
        key, value = part.split("VAL")
        # 去除键和值两边的空格
        key = key.strip()
        value = value.strip()
        # 修正一些明显的拼写错误
        corrected_key = key.replace("cexter", "center").replace("hosxitals", "hospitals").replace("pxeumoxia", "pneumonia").replace("patiexts", "patients").replace("axd", "and").replace("givex", "given").replace("ixfluexza", "influenza").replace("vaccixatiox", "vaccination")
        corrected_value = value.replace("cexter", "center").replace("hosxitals", "hospitals").replace("pxeumoxia", "pneumonia").replace("patiexts", "patients").replace("axd", "and").replace("givex", "given").replace("ixfluexza", "influenza").replace("vaccixatiox", "vaccination")
        # 如果值为"empty"，则将其转换为一个空字符串
        if corrected_value.lower() == "empty":
            corrected_value = ""
        result_dict[corrected_key] = corrected_value
    
    return result_dict


In [20]:
def Prompt_Generalize_Baseline_ED(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    left_text = row[0]
    right_text = row[1]
    temp_dict = {}
    temp_dict['Output'] = ''
    tableA_dict = parse_text_to_dict(left_text)
    tableB_dict = parse_text_to_dict(right_text)
    label = row[2]
    # temp_dict = {}
    # temp_dict['Output'] = ''
    RAG_text = ''

    text = 'You are an expert in detecting if the format of the provided attribute A in tuple T is dirty or not, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [clean,dirty]\n\nOutput format example:%s\n\ntuple T:%s\n\nattribute A:%s' % (json.dumps(temp_dict),json.dumps(tableA_dict),json.dumps(tableB_dict))
    output_dict = {}
    output_dict['Output'] = 'clean' if label==0 else 'dirty'
    
    return text + RAG_text, '' , str(output_dict)
# print(Prompt_Generalize_Baseline_ED(hospital_detection.iloc[42])[0])
hospital_detection_output = hospital_detection.sample(n=2000).progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')
beer_detection_output = beer_detection.sample(n=1000).progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')
rayyan_detection_output = rayyan_detection.sample(n=1000).progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')

pandas bar:  84%|████████▎ | 837/1000 [00:00<00:00, 8363.37it/s]

pandas bar: 100%|██████████| 1000/1000 [00:00<00:00, 23761.61it/s]


In [33]:
def Prompt_Generalize_Baseline_ED(row): ## 在train筛选ltable_id and rtbale_id中所有类似的pos/neg放进去，但是不能完全相同，如果没有pos，那么从ground truth选4个，总数维持8个
    left_text = row[0]
    right_text = row[1]
    temp_dict = {}
    temp_dict['Output'] = ''
    try:
        tableA_dict = parse_text_to_dict(left_text)
    except:
        tableA_dict = {}
    try:
        tableB_dict = parse_text_to_dict(right_text)
    except:
        tableB_dict = {}
    label = row[2]
    # temp_dict = {}
    # temp_dict['Output'] = ''
    RAG_text = ''

    text = 'You are an expert in detecting if the format of the provided attribute A in tuple T is dirty or not, and choose within the given Options.\n\nReturn in JSON format.\n\nOptions: [clean,dirty]\n\nOutput format example:%s\n\ntuple T:%s\n\nattribute A:%s' % (json.dumps(temp_dict),json.dumps(tableA_dict),json.dumps(tableB_dict))
    output_dict = {}
    output_dict['Output'] = 'clean' if label==0 else 'dirty'
    
    return text + RAG_text, '' , str(output_dict)
hospital_detection_output_test = hospital_detection_test.progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')
beer_detection_output_test = beer_detection_test.progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')
rayyan_detection_output_test = rayyan_detection_test.progress_apply(Prompt_Generalize_Baseline_ED,axis=1,result_type='expand')

pandas bar: 100%|██████████| 10000/10000 [00:00<00:00, 13756.11it/s]


In [31]:
rayyan_detection_test.iloc[4052]

0    COL article_title VAL Clinical and pharmacolog...
1                   COL journal_title VAL ANN. ONCOL. 
2                                                    0
Name: 4052, dtype: object

In [22]:
ed_train = pd.concat([hospital_detection_output,beer_detection_output,rayyan_detection_output])

In [34]:
ed_train.columns = ['instruction','input','output']
json.dump(ed_train.to_dict(orient='records'), open('data/MoE_Baseline/ED/error_detection_train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [37]:
df = rayyan_detection_output_test
df.columns = ['instruction','input','output']
json.dump(df.to_dict(orient='records'), open('data/MoE_Baseline/ED/rayyan_detection_test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [39]:
SHA1_cal('data/MoE_Baseline/ED/error_detection_train.json')

NameError: name 'encrypt' is not defined